In [4]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [1]:
from typing import List
from pydantic import BaseModel, Field

Pydantic Syntax
Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic.
They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.
In standard python you would create a class like this:

In [18]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [19]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [20]:
foo.name

'Joe'

In [21]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [22]:
foo.age

'bar'

bar is not an integer. If it says it's an integer, it should throw up an error. Pydantic will do it

In [23]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [24]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [25]:
foo_p.name

'Jane'

In [14]:
#foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com") #Expected to generate error

In [26]:
class Class(BaseModel):
    students: List[pUser]

Data can be nested

In [27]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [29]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

Pydantic to OpenAI function definition

In [47]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [48]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

Converts Pydantic to JSON format required for OpenAI function

In [52]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [53]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [58]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [59]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [55]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [56]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

Now, lets combine openai functions with langchain expression language

In [61]:
from langchain_openai import ChatOpenAI

In [62]:
model = ChatOpenAI()

In [71]:
model.invoke("what is the weather in SF today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 70, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-e3c78409-7819-411d-9022-928b088eb606-0', usage_metadata={'input_tokens': 70, 'output_tokens': 17, 'total_tokens': 87})

In [72]:
model_with_function = model.bind(functions=[weather_function])

In [67]:
model_with_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-adea45c9-74fc-4208-8c1b-a185b4dd0238-0', usage_metadata={'input_tokens': 69, 'output_tokens': 17, 'total_tokens': 86})